This is the jupiter notebook that Zhou writes some test functions

In [2]:
from functions import *
from optimize_obj import *
from Optalg import *

In [3]:
caseid_list = [1,2,3,4,5,6,7,8,9,10,11,12]
objname_list = ["Rosenbrock 1", "Rosenbrock 2", "Rosenbrock 3", "Rosenbrock 4", "Rosenbrock 5", "Rosenbrock 6", "Rosenbrock 7", "Beale Function 1", "Beale Function 2", "Problem 10"]
dim_list = [10, 10, 1000, 1000, 1000, 1000, 1000, 2, 2, 10]
maxiter_list = [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
xinit_list = []
for i in range(10):
    if i == 0:
        xinit_list.append(np.array([-1.2, 1]))
    elif i == 1 or i == 3:
        dim = dim_list[i]
        xinit_list.append(-1 * np.ones(dim))
    elif i == 2 or i == 4 or i == 5 or i == 6:
        dim = dim_list[i]
        xinit_list.append(2 * np.ones(dim))
    elif i == 7:
        xinit_list.append(np.array([1, 1]))
    elif i == 8:
        xinit_list.append(np.array([0, 0]))
    elif i == 9:
        xinit_list.append(np.array([1,2,3,4,5,6,7,8,9,10]))

In [4]:
def runcase(Obj, xinit, Objname = 'OptObj', line_search_method = 'Armijo', methodlist = ['gradient_descend', 'BFGS','L-BFGS','Newton-CG', 'modified_Newton'], make_table = True):
    # Run the optimization for one Obj using three methods: gradient descend, Newton, modified Newton
    
    # Create an outputfile with name Objname, in subdirectory '/result'
    
    # Gradient descend
    myobj = Obj

    outputfile_list = []    
    outinfo_list = []
    for method in methodlist:
        
        myobj.method = method
        myobj.linesearch_options.method = line_search_method 
        
        if line_search_method == 'Armijo':
            outputfile = 'result/' + Objname + '_' + method + '_Armijo.txt'
        elif line_search_method == 'Wolfe':
            outputfile = 'result/' + Objname + '_' + method + '_Wolfe.txt'
        Obj.outputfile = outputfile
        myobj.printinfo()
        
        xopt, outinfo = OptAlg(myobj, xinit, outputfile)
        
        outputfile_list.append(outputfile)
        outinfo_list.append(outinfo)
        
    if make_table:
        # Print the code for a LaTeX table, with caption Objname.
        # Save the number with .6 digit scientific notation
        # Print the converge rate with .3 digit scientific notation
        # Print the runtime with .3 digit scientific notation
        # Print the number of iterations
        # Print the reason for convergence
        # Print convergence rate


        print("\\begin{table}[htpb]")
        print("\\centering")
        print("\\caption{" + Objname + "}")
        print("\\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}")
        print("\\hline")
        print(" Method & $f_{\\text{val}}$ & $||\\nabla f||$ &  Iter & Runtime & Converge Reason & Other info \\\\")
        print("\\hline")
        
        
        for outinfo in outinfo_list:
            method = outinfo.method
            fval = outinfo.fval
            gradnorm = outinfo.gradnorm
            iternum = outinfo.iternum
            runtime = outinfo.runtime
            converge_reason = outinfo.converge_reason
            otherinfo = outinfo.other
            if otherinfo is None:
                otherinfo = '/'
            else:
                otherinfo = str(otherinfo)
            
            print(method + " & " + "{:.5e}".format(fval) + " & " + "{:.5e}".format(gradnorm) + " & " + str(iternum) + " & " + "{:.3e}".format(runtime) + " & " + converge_reason + " & " + otherinfo + " \\\\")
        
        print("\\hline")
        print("\\end{tabular}")
        print("\\end{table}")
        
    
    # Write the latex table code in one txt file
    with open('result/' + Objname + '_table.txt', 'w') as f:
        f.write("\\begin{table}[htpb]\n")
        f.write("\\centering\n")
        f.write("\\caption{" + Objname + "}\n")
        f.write("\\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|}\n")
        f.write("\\hline\n")
        f.write(" Method & $f_{\\text{val}}$ & $||\\nabla f||$ &  Iter & Runtime & Converge Reason & Other info \\\\\n")
        f.write("\\hline\n")
        
        for outinfo in outinfo_list:
            method = outinfo.method
            fval = outinfo.fval
            gradnorm = outinfo.gradnorm
            iternum = outinfo.iternum
            runtime = outinfo.runtime
            converge_reason = outinfo.converge_reason
            otherinfo = outinfo.other
            if otherinfo is None:
                otherinfo = '/'
            else:
                otherinfo = str(otherinfo)
            
            f.write(method + " & " + "{:.5e}".format(fval) + " & " + "{:.5e}".format(gradnorm) + " & " + str(iternum) + " & " + "{:.3e}".format(runtime) + " & " + converge_reason + " & " + otherinfo + " \\\\\n")
        
        f.write("\\hline\n")
        f.write("\\end{tabular}\n")
        f.write("\\end{table}\n")
    

This is for the problem 1

In [6]:
# dimension n = 10
n = 10
# randomly generate a symmetric positive definite matrix Q, and q
Q = np.random.rand(n, n)
Q = Q @ Q.T
q = np.random.rand(n)

# declare the object of the class optimize_obj
obj = quad_obj(Q, q)

# solve the objective function using the gradient descent method
x0 = np.random.rand(n)

runcase(obj, x0, Objname = 'Quadratic', line_search_method = 'Wolfe', make_table = True)

Objective function info:
problem dim =  10
method =  gradient_descend
Line search options:
Line search method =  Wolfe
alphainit =  1
beta =  0.5
c1 =  0.0001
c2 =  0.9
minstep =  1e-13
Convergence options:
ftol =  1e-10
gtol =  1e-10
xtol =  1e-10
maxiter =  1000
Iter	fval		||grad||	alpha
0	14.93364659	28.47907066	0.00000000
[10.04747804  8.10138019  8.76838592  9.0745996   9.49068568  8.07011637
  9.15493628 11.45458275  6.98209953  8.14066391]
[-10.04747804  -8.10138019  -8.76838592  -9.0745996   -9.49068568
  -8.07011637  -9.15493628 -11.45458275  -6.98209953  -8.14066391]
1	7.22437157	19.96348785	0.06250000
[-5.37310443 -6.05792073 -6.10125467 -6.49743958 -7.49217065 -6.92189959
 -5.87000632 -7.92574952 -5.03435171 -5.18321049]
[5.37310443 6.05792073 6.10125467 6.49743958 7.49217065 6.92189959
 5.87000632 7.92574952 5.03435171 5.18321049]
2	3.40908413	14.00004066	0.06250000
[5.21079855 3.99633668 4.37991202 4.40807003 4.5142113  3.73321867
 4.46094306 5.62694296 3.48741651 4.02786